In [1]:
from src.KNN.ItemKNNCBFCFSimilarityHybridRecommender import ItemKNNCBFCFSimilarityHybridRecommender
from src.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Base.Evaluation.Evaluator import EvaluatorHoldout
from src.Utils.load_URM import load_URM
from src.Utils.load_ICM import load_ICM
from operator import itemgetter
import numpy as np

URM_all = load_URM("in/data_train.csv")
ICM_all = load_ICM("in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [2]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [3]:
recommender = ItemKNNCBFCFSimilarityHybridRecommender(
    URM_train=URM_train,
    ICM_train=ICM_all,
    topK_knncf=100,
    shrink_knncf=50,
    topK_knncbf=500,
    shrink_knncbf=10
)

recommender.fit(topK=500, alpha=0.67)

URM_estimated = URM_train.dot(recommender.W_sparse)

ItemKNNCFRecommender: URM Detected 64 (0.81 %) cold users.
ItemKNNCFRecommender: URM Detected 2402 (9.25 %) cold items.
Similarity column 25975 ( 100 % ), 7273.92 column/sec, elapsed time 0.06 min
ItemKNNCBFRecommender: URM Detected 64 (0.81 %) cold users.
ItemKNNCBFRecommender: URM Detected 2402 (9.25 %) cold items.
Similarity column 25975 ( 100 % ), 4288.34 column/sec, elapsed time 0.10 min
 ItemKNNCBFCFSimilarityHybridRecommender: URM Detected 64 (0.81 %) cold users.
 ItemKNNCBFCFSimilarityHybridRecommender: URM Detected 2402 (9.25 %) cold items.


In [4]:
MAP=[]

for topK in [10, 50, 100, 200, 500]:
    for num_factors in [100, 500, 600, 700, 800]:
        recommender = PureSVDItemRecommender(URM_estimated)
        recommender.fit(
            num_factors=num_factors,
            topK= topK
        )

        recommender.URM_train=URM_train

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

        MAP.append(('topKo={}'.format(topK),'num_factors={}'.format(num_factors),result_dict[10]["MAP"]))

        print(MAP[-1])


PureSVDItemRecommender: URM Detected 64 (0.81 %) cold users.
PureSVDItemRecommender: URM Detected 5 (0.02 %) cold items.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 5765 ( 100.00% ) in 3.80 sec. Users per second: 1516
('topKo=10', 'num_factors=100', 0.02389131811216023)
PureSVDItemRecommender: URM Detected 64 (0.81 %) cold users.
PureSVDItemRecommender: URM Detected 5 (0.02 %) cold items.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 5765 ( 100.00% ) in 3.75 sec. Users per second: 1538
('topKo=10', 'num_factors=500', 0.032895123542012614)
PureSVDItemRecommender: URM Detected 64 (0.81 %) cold users.
PureSVDItemRecommender: URM Detected 5 (0.02 %) cold items.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Pr

KeyboardInterrupt: 

In [ ]:
max_tuple = max(MAP, key=itemgetter(2))
print('The best combination for the PureSVD recommender is ({},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1],max_tuple[2]))